In [ ]:
import time
import sys
import pandas as pd
import google.generativeai as genai
from datasets import load_dataset
from datasets import Dataset

the_key = input("input your gemini key: ")
genai.configure(api_key=the_key)
model = genai.GenerativeModel(model_name="models/gemini-1.5-flash")

# 라벨링 질문 목록
LABEL_QUESTIONS = {
  "abstract": "Is this sentence abstract or concrete? Respond with 1 if abstract, 0 if concrete.",
  "objective": "Is this sentence objective or subjective? Respond with 1 if objective, 0 if subjective.",
  "rational": "Is this sentence rational or emotional? Respond with 1 if rational, 0 if emotional."
}

def ask_gemini(prompt):
    E = None
    for attempt in range(16):
        try:
            response = model.generate_content(prompt)
            time.sleep(4.2)
            return int(response.text.strip()[0])
        except Exception as e:
            print("error count", attempt)
            E = e
            time.sleep(attempt + 0.5)
    print("fail to request")
    print("Gemini Error:", E)
    sys.exit(1)
    

def generate_dataset(sentences):
    count = len(sentences["sentence"])
    for i in range(count):
        prompt = build_label_prompts(sentences["sentence"][i], LABEL_QUESTIONS)
        abst = ask_gemini(prompt["abstract"])
        obje = ask_gemini(prompt["objective"])
        rati = ask_gemini(prompt["rational"])
        sentences["label"][i][1] = abst
        sentences["label"][i][2] = obje
        sentences["label"][i][3] = rati


def build_label_prompts(sentence, label_questions):
    prompts = {}
    for label, question in label_questions.items():
        prompt = f"Sentence: \"{sentence}\"\n{question}"
        prompts[label] = prompt
    return prompts

def convert_label(row):
    return [row["label"], 0, 0, 0]


#데이터 불러오기
dataset = load_dataset("glue", "sst2")["train"].to_pandas()

#데이터 일부만 
generating_count = int(input("input number of count of generating data: "))
small_data = dataset[0:generating_count].copy()

#라벨 가공
small_data["label"] = small_data.apply(convert_label, axis=1)

generate_dataset(small_data)
print(small_data)

new_dataset = Dataset.from_pandas(small_data)
new_dataset.to_csv("my_dataset.csv")

print("end")